Это данные задачи двоичной классификации в формате табличных данных .
Полис рекомендуется человеку, когда он попадает на страховой веб-сайт, и если человек решает заполнить форму для подачи заявки, это считается положительным результатом (классифицируется как лид). Все остальные условия считаются нулевыми исходами.
- Задача найти других потенциальных лидов (увеличить recall).
- https://www.kaggle.com/datasets/owaiskhan9654/health-insurance-lead-prediction-raw-data

In [171]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import dill

In [173]:
df = pd.read_csv('Health Insurance Lead Prediction Raw Data.csv', sep=',')
df

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,50878,C4,845,Rented,Individual,22,22,No,X3,NaN,NaN,18,7704.0,0
50878,50879,C5,4188,Rented,Individual,27,27,No,X3,7.0,3.0,4,5408.0,0
50879,50880,C1,442,Rented,Individual,63,63,No,X2,14+,1.0,12,11374.0,0
50880,50881,C1,4,Owned,Joint,71,49,No,X2,2.0,2.0,16,28179.2,1


In [174]:
df['Response'].value_counts()

0    38673
1    12209
Name: Response, dtype: int64

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       50882 non-null  int64  
 1   City_Code                50882 non-null  object 
 2   Region_Code              50882 non-null  int64  
 3   Accomodation_Type        50882 non-null  object 
 4   Reco_Insurance_Type      50882 non-null  object 
 5   Upper_Age                50882 non-null  int64  
 6   Lower_Age                50882 non-null  int64  
 7   Is_Spouse                50882 non-null  object 
 8   Health Indicator         39191 non-null  object 
 9   Holding_Policy_Duration  30631 non-null  object 
 10  Holding_Policy_Type      30631 non-null  float64
 11  Reco_Policy_Cat          50882 non-null  int64  
 12  Reco_Policy_Premium      50882 non-null  float64
 13  Response                 50882 non-null  int64  
dtypes: float64(2), int64(6

In [176]:
df.describe()

,ID,Region_Code,Upper_Age,Lower_Age,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
count,50882.000000,50882.000000,50882.000000,50882.000000,30631.000000,50882.000000,50882.000000,50882.000000
mean,25441.500000,1732.788707,44.856275,42.738866,2.439228,15.115188,14183.950069,0.239947
std,14688.512535,1424.081652,17.310271,17.319375,1.025923,6.340663,6590.074873,0.427055
min,1.000000,1.000000,18.000000,16.000000,1.000000,1.000000,2280.000000,0.000000
25%,12721.250000,523.000000,28.000000,27.000000,1.000000,12.000000,9248.000000,0.000000
50%,25441.500000,1391.000000,44.000000,40.000000,3.000000,17.000000,13178.000000,0.000000
75%,38161.750000,2667.000000,59.000000,57.000000,3.000000,20.000000,18096.000000,0.000000
max,50882.000000,6194.000000,75.000000,75.000000,4.000000,22.000000,43350.400000,1.000000


## Разбиение на train и test

In [177]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Response'], axis=1), df['Response'], test_size=0.33, random_state=42)

X_train.to_csv('X_train.csv', index=None)
y_train.to_csv('y_train.csv', index=None)

X_test.to_csv('X_test.csv', index=None)
y_test.to_csv('y_test.csv', index=None)

## Сбор пайплайна и feature engineering

In [178]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.column]
        

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []
    
    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self
    
    def transform(self, X, y=None):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


class Numericlog(BaseEstimator, TransformerMixin):
    def __init__(self, key, p):
        self.key = key
        self.columns = []
        self.p = p + 1
    
    def fit(self, X, y=None):
        for i in range(2, self.p):
            self.columns += [self.key+'log_'+str(i)]
        self.columns += ['sin_'+self.key]
        self.columns += ['cos_'+self.key]
        self.columns += ['tg_'+self.key]
        return self
    
    def transform(self, X, y=None):
        Xp = X.copy()
        
        for i in range(2, self.p):
            Xp[self.key+'log_'+str(i)] = np.log(np.abs(Xp[self.key].values)+0.01)/np.log(i)
        Xp['sin_'+self.key] = np.sin(Xp[self.key].values)
        Xp['cos_'+self.key] = np.cos(Xp[self.key].values)
        Xp['tg_'+self.key] = np.tan(Xp[self.key].values)
        return Xp.loc[:, self.columns]


class NumericMix(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []
    
    def fit(self, X, y=None):
        for col1 in self.key[:-1]:
            for col2 in self.key[1:]:
                self.columns += [col1+'/'+col2, col1+'*'+col2]
        return self
    
    def transform(self, X, y=None):
        Xp = X[self.key].copy()
        
        for col1 in self.key[:-1]:
            for col2 in self.key[1:]:
                Xp[col1+'/'+col2] = Xp.loc[:, col1].values/(np.abs(Xp.loc[:, col2].values) + 0.01)
                Xp[col1+'*'+col2] = Xp.loc[:, col1].values*Xp.loc[:, col2].values
        
        return Xp[self.columns]

class CleanData(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for i in self.columns:
            X[i] = X[i].fillna(0)
        X['Reco_Insurance_Type'] = X['Reco_Insurance_Type'].replace({'Individual': 0, 'Joint': 1})

        Hind = {f'X{i}': i for i in range(1, 10)}
        X['Health Indicator'] = X['Health Indicator'].replace(Hind)

        Ccode = {f'C{i}': i for i in range(1, 37)}
        X['City_Code'] = X['City_Code'].replace(Ccode)

        types = {'Reco_Policy_Cat': 'int64', 
                 'City_Code': 'int16',
                 'Region_Code': 'int64',
                 'Holding_Policy_Type': 'float16',
                 'Reco_Insurance_Type': 'int8',
                 'Health Indicator': 'int16'}
        for k, v in types.items():
            X[k] = X[k].astype(v)
        return X[self.columns]

In [179]:
continuous_columns = ['Reco_Policy_Cat', 'City_Code', 'Region_Code']
categorical_columns = ['Holding_Policy_Type', 'Reco_Insurance_Type', 'Health Indicator']

In [180]:
final_transformers = []

numix = Pipeline([
    ('selector', FeatureSelector(column=continuous_columns)),
    ('numerMix', NumericMix(key=continuous_columns)),
    ('scaler', StandardScaler())
    ])
final_transformers.append(('numerMix', numix))

for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('numlog', Numericlog(key=cont_col, p=10)),
        ('scaler', StandardScaler())
    ])
    final_transformers.append((cont_col, cont_transformer))

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
        ('selector', FeatureSelector(column=cat_col)),
        ('scaler', OHEEncoder(key=cat_col))
    ])
    final_transformers.append((cat_col, cat_transformer))

In [181]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Добавление классификатора

In [182]:
pipeline = Pipeline([
        ('cleaner', CleanData(columns=continuous_columns+categorical_columns)),
        ('feats', feats),
        ('classifier', GradientBoostingClassifier(max_depth=3, n_estimators=250))
])

In [183]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('cleaner',
                 CleanData(columns=['Reco_Policy_Cat', 'City_Code',
                                    'Region_Code', 'Holding_Policy_Type',
                                    'Reco_Insurance_Type',
                                    'Health Indicator'])),
                ('feats',
                 FeatureUnion(transformer_list=[('numerMix',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column=['Reco_Policy_Cat',
                                                                                          'City_Code',
                                                                                          'Region_Code'])),
                                                                 ('numerMix',
                                                                  NumericMix(key=['Reco_Policy_Cat...
                                               

Сохранение модели

In [184]:
with open("GradientBoostModel.dill", "wb") as f:
    dill.dump(pipeline, f)

Проверка пайплайна

In [185]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [190]:
X_test

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,19809,C4,3283,Rented,Individual,25,25,No,X2,NaN,NaN,14,7068.0
1,17701,C4,1308,Rented,Individual,23,23,No,X1,NaN,NaN,22,4080.0
2,24040,C2,168,Rented,Joint,26,24,Yes,X2,1.0,3.0,13,16486.4
3,23030,C2,256,Rented,Individual,54,54,No,X4,11.0,1.0,22,19214.0
4,44333,C7,4006,Owned,Individual,75,75,No,X3,3.0,3.0,21,20376.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16787,2897,C1,279,Owned,Joint,69,64,Yes,X1,NaN,NaN,18,30345.0
16788,44869,C16,404,Owned,Joint,23,19,Yes,X1,NaN,NaN,19,15299.2
16789,42159,C8,1856,Rented,Individual,19,19,No,X3,NaN,NaN,21,4170.0
16790,31489,C19,3234,Rented,Individual,28,28,No,X1,NaN,NaN,2,5930.0


In [191]:
with open("GradientBoostModel.dill", "rb") as m:
    pipeline = dill.load(m)

In [192]:
pipeline

Pipeline(steps=[('cleaner',
                 CleanData(columns=['Reco_Policy_Cat', 'City_Code',
                                    'Region_Code', 'Holding_Policy_Type',
                                    'Reco_Insurance_Type',
                                    'Health Indicator'])),
                ('feats',
                 FeatureUnion(transformer_list=[('numerMix',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column=['Reco_Policy_Cat',
                                                                                          'City_Code',
                                                                                          'Region_Code'])),
                                                                 ('numerMix',
                                                                  NumericMix(key=['Reco_Policy_Cat...
                                               

In [193]:
pred = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'pred': pred})
pred_df.to_csv('test_pred.csv', index=None)

In [194]:
precision, recall, thresholds = precision_recall_curve(y_test, pred)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Thresholds=%.3f, Precision=%.3f, Recall=%.3f, F-Score=%.3f' % (thresholds[ix],
                                                     precision[ix],
                                                     recall[ix],
                                                     fscore[ix]))

Thresholds=0.228, Precision=0.325, Recall=0.737, F-Score=0.451


In [198]:
from flask import Flask, request, jsonify


In [219]:
app = Flask(__name__)

@app.route("/", methods=["GET"])
def hello():
    return 'Welcome to prediction lead for insuarance company'

@app.route("/predict", methods=['POST'])
def predict():
    data = {'success': False}

    City_Code, Region_Code, Reco_Policy_Cat, Reco_Insurance_Type, Holding_Policy_Type, Health_Indicator = 0, 0, 0, 0, 0, 0
    
    request_json = request.get_json()

    if request_json['City_Code']:
        City_Code = request_json['City_Code']
    
    if request_json['Reco_Policy_Cat']:
        Reco_Policy_Cat = request_json['Reco_Policy_Cat']

    if request_json['Region_Code']:
        Region_Code = request_json['Region_Code']
    
    if request_json['Holding_Policy_Type']:
        Holding_Policy_Type = request_json['Holding_Policy_Type']

    if request_json['Reco_Insurance_Type']:
        Reco_Insurance_Type = request_json['Reco_Insurance_Type']
    
    if request_json['Health Indicator']:
        Health_Indicator = request_json['Health Indicator']

    pred = pipeline.predict_proba(pd.DataFrame({'City_Code': [City_Code],
                                                'Region_Code': [Region_Code],
                                                'Reco_Policy_Cat': [Reco_Policy_Cat],
                                                'Reco_Insurance_Type': [Reco_Insurance_Type],
                                                'Holding_Policy_Type': [Holding_Policy_Type],
                                                'Health Indicator': [Health_Indicator]}))
    
    data['pred_proba'] = pred[:, 1][0]
    data['Response'] = bool(pred[:, 1][0] > thresholds[ix])
    data['success'] = True
    print('OK')

    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://89f8-35-199-166-129.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Nov/2022 14:17:07] "POST /predict HTTP/1.1" 200 -


{'City_Code': 'C4', 'Region_Code': 3283, 'Reco_Policy_Cat': 14, 'Reco_Insurance_Type': 'Individual', 'Holding_Policy_Type': nan, 'Health Indicator': 'X2'}
FeatureUnion(transformer_list=[('numerMix',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column=['Reco_Policy_Cat',
                                                                         'City_Code',
                                                                         'Region_Code'])),
                                                ('numerMix',
                                                 NumericMix(key=['Reco_Policy_Cat',
                                                                 'City_Code',
                                                                 'Region_Code'])),
                                                ('scaler', StandardScaler())])),
                               ('Reco_Policy_Cat',
                                P

...

FeatureUnion(transformer_list=[('numerMix',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column=['Reco_Policy_Cat',
                                                                         'City_Code',
                                                                         'Region_Code'])),
                                                ('numerMix',
                                                 NumericMix(key=['Reco_Policy_Cat',
                                                                 'City_Code',
                                                                 'Region_Code'])),
                                                ('scaler', StandardScaler())])),
                               ('Reco_Policy_Cat',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='Reco_Policy_Cat')),
                                      

INFO:werkzeug:127.0.0.1 - - [24/Nov/2022 14:28:24] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2022 14:28:24] "POST /predict HTTP/1.1" 200 -


[[0.73720734 0.26279266]]
OK
{'City_Code': 'C1', 'Region_Code': 622, 'Reco_Policy_Cat': 19, 'Reco_Insurance_Type': 'Individual', 'Holding_Policy_Type': nan, 'Health Indicator': 'X3'}
FeatureUnion(transformer_list=[('numerMix',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column=['Reco_Policy_Cat',
                                                                         'City_Code',
                                                                         'Region_Code'])),
                                                ('numerMix',
                                                 NumericMix(key=['Reco_Policy_Cat',
                                                                 'City_Code',
                                                                 'Region_Code'])),
                                                ('scaler', StandardScaler())])),
                               ('Reco_Policy_Cat',
     

INFO:werkzeug:127.0.0.1 - - [24/Nov/2022 14:33:32] "POST /predict HTTP/1.1" 200 -


{'City_Code': 'C4', 'Region_Code': 3283, 'Reco_Policy_Cat': 14, 'Reco_Insurance_Type': 'Individual', 'Holding_Policy_Type': nan, 'Health Indicator': 'X2'}
FeatureUnion(transformer_list=[('numerMix',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column=['Reco_Policy_Cat',
                                                                         'City_Code',
                                                                         'Region_Code'])),
                                                ('numerMix',
                                                 NumericMix(key=['Reco_Policy_Cat',
                                                                 'City_Code',
                                                                 'Region_Code'])),
                                                ('scaler', StandardScaler())])),
                               ('Reco_Policy_Cat',
                                P